In [41]:
import requests
from bs4 import BeautifulSoup
import collections
import itertools
import re
import pandas as pd
import numpy as np

In [21]:
chipotle_locations = requests.get('https://locations.chipotle.com/index.html')

In [22]:
chipotle_locations.headers

{'Date': 'Sun, 28 Jul 2019 15:20:17 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Set-Cookie': '__cfduid=d44eda401d91c1b5c00eb58dec578674f1564327217; expires=Mon, 27-Jul-20 15:20:17 GMT; path=/; domain=.locations.chipotle.com; HttpOnly', 'Content-Encoding': 'gzip', 'Etag': '"27f0b43fb9278e77cab3b2c0ce59cd65"-gzip', 'Last-Modified': 'Fri, 26 Jul 2019 21:42:16 GMT', 'Surrogate-Key': 'locations.chipotle.com, locations.chipotle.com/index.html', 'Vary': 'Accept-Encoding', 'X-Amz-Id-2': '7H/rG1TanYMaRVbmlljF8OgsePwychTjeKlmyZOwFTXL9SqxEy2EVW024auC4qjT8cs49fBCg9A=', 'X-Amz-Request-Id': '461E83E08AC3F798', 'X-Amz-Server-Side-Encryption': 'AES256', 'X-Amz-Version-Id': 'null', 'X-Yext-Site': 'us2', 'CF-Cache-Status': 'HIT', 'Age': '4236', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Server': 'cloudflare', 'CF-RAY': '4fd7d61699e493ee-SJC'}

In [23]:
?chipotle_locations

Type:        Response
String form: <Response [200]>
File:        c:\users\dinel\anaconda3\lib\site-packages\requests\models.py
Docstring:  
The :class:`Response <Response>` object, which contains a
server's response to an HTTP request.


In [24]:
?bs4

Object `bs4` not found.


In [25]:
c = chipotle_locations.content

In [26]:
soup = BeautifulSoup(c, 'html.parser')

In [27]:
states = soup.findAll("a", {"class":"Directory-listLink"})

In [28]:
states_dict = {}
states_dc = {}
for state in states:
    state_name = state.string
    state_link = 'https://locations.chipotle.com/' + state.attrs['href']
    data_count = int(str(state.attrs['data-count']).replace('(','').replace(')',''))
    states_dict[state_name] = state_link
    states_dc[state_name] = data_count

In [29]:
locations_dict = {}
all_cities = []
all_links = []
all_city_counts = {}
for state, link in states_dict.items():
    c = requests.get(link).content
    soup = BeautifulSoup(c)
    cities = soup.findAll('a', {'class':'Directory-listLink'})
    city_list = [city.string for city in cities]
    url_list = [link[:-2] + city.attrs['href'] for city in cities]
    shop_counts_per_city = [int(city.attrs['data-count'].replace('(', '').replace(')', '')) for city in cities]
    city_url = dict(zip(city_list, url_list))
    city_shop_count = dict(zip(city_list, shop_counts_per_city))
    all_city_counts[state] = city_shop_count
    locations_dict[state] = city_url
    all_cities.extend(city_list)
    all_links.extend(url_list)
        
        

all_city_urls = dict(zip(all_cities, all_links))

In [30]:
all_city_counts['Mississippi'] = {'Oxford': 1}
all_city_counts['North Dakota'] = {'Fargo': 1}
all_city_counts['Vermont'] = {'Burlington': 1}
all_city_counts['Washington DC'] = {'Washington DC': 21}
all_city_counts['Wyoming'] = {'Cheyenne': 1}    

In [31]:
# Used to correct an error on the small amount of states with either one location or one city like Washington D.C
locations_dict['Mississippi'] = {'Oxford':'https://locations.chipotle.com/ms/oxford/2151-jackson-ave-w'}
locations_dict['North Dakota'] = {'Fargo':'https://locations.chipotle.com/nd/fargo/1680-45th-st-s'}
locations_dict['Vermont'] = {'Burlington': 'https://locations.chipotle.com/vt/burlington/580-shelburne-rd'}
locations_dict['Washington DC'] = {'Washington DC': 'https://locations.chipotle.com/dc/washington'}
locations_dict['Wyoming'] = {'Cheyenne': 'https://locations.chipotle.com/wy/cheyenne/1508-dell-range-blvd'}

In [32]:
r = requests.get(states_dict['Washington DC']).content
soup = BeautifulSoup(r)
num_stores = int(str(soup.find('h1', {'class': 'Directory-title'}).string)[0:3].strip())
print(num_stores)

21


In [33]:
t = list(all_city_urls.items())[0]
q = requests.get(t[1]).content
soup = BeautifulSoup(q)
locdat = soup.findAll('span', {'class':'coordinates'})
locdat

[<span class="coordinates" itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates"><meta content="38.94120080026161" itemprop="latitude"/><meta content="-121.09614846331169" itemprop="longitude"/></span>,
 <span class="coordinates" itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates"><meta content="38.94120080026161" itemprop="latitude"/><meta content="-121.09614846331169" itemprop="longitude"/></span>]

In [34]:

r = requests.get(states_dict['Alabama']).content
soup = BeautifulSoup(r)
loc = soup.findAll('a', {'class': 'Directory-listLink'})

    

In [35]:
one_shop_cities = collections.defaultdict(list)
for state in all_city_counts.keys():
    for city, count in all_city_counts[state].items():
        if count == 1:
            one_shop_cities[state].append(city)


'Periora' in one_shop_cities

False

In [36]:
for city, link in locations_dict['Arizona'].items():
    r = requests.get(link).content
    soup = BeautifulSoup(r)
    if city not in one_shop_cities['Arizona']:
        num_stores = all_city_counts['Arizona'][city]
        store_titles = [store.string for store in soup.findAll('span', {'class': 'LocationName'})]
        store_links = ['https://locations.chipotle.com' + str(loclink.attrs['href']).replace('..','') for loclink in soup.findAll('a', {'class': 'Teaser-titleLink'})]
        if num_stores != len(store_titles) or num_stores != len(store_links):
            print(city, num_stores, store_titles, store_links)
            print('\n')

In [37]:
all_city_counts['California']['San Rafael']

2

In [39]:
locations_csv_frame = [['Store Title', 'State', 'City', 'Latitude', 'Longitude']] 
for state in locations_dict.keys():
    for city, link in locations_dict[state].items():
        r = requests.get(link).content
        soup = BeautifulSoup(r)
        if city in one_shop_cities[state]:
            latitude = soup.find('meta', {'itemprop': 'latitude'}).attrs['content']
            longitude = soup.find('meta', {'itemprop': 'longitude'}).attrs['content']
            store_title = soup.find('h1', {'class': 'Hero-title'}).string
            locations_csv_frame.append([store_title, state, city, latitude, longitude])
        else:
            num_stores = all_city_counts[state][city]
            store_titles =[(title.string) for title in soup.findAll('span', {'class': 'LocationName'})]
            store_links = ['https://locations.chipotle.com' + str(loclink.attrs['href']).replace('..', '') for loclink in soup.findAll('a', {'class': 'Teaser-titleLink'})]
            for i in range(num_stores):
                store_r = requests.get(store_links[i]).content
                store_soup = BeautifulSoup(store_r)
                latitude = store_soup.find('meta', {'itemprop': 'latitude'}).attrs['content']
                longitude = store_soup.find('meta', {'itemprop': 'longitude'}).attrs['content']
                locations_csv_frame.append([store_titles[i], state, city, latitude, longitude])
                
                
        
         

[['Store Title', 'State', 'City', 'Latitude', 'Longitude'],
 ['Chipotle Auburn Campus',
  'Alabama',
  'Auburn',
  '32.606812966051244',
  '-85.48732833164195'],
 ['Chipotle UAB Birmingham',
  'Alabama',
  'Birmingham',
  '33.509721495414745',
  '-86.80275567068401'],
 ['Chipotle Trussville',
  'Alabama',
  'Birmingham',
  '33.59558141391436',
  '-86.64743684970284'],
 ['Chipotle Inverness',
  'Alabama',
  'Birmingham',
  '33.42258214624579',
  '-86.69827946502971'],
 ['Chipotle Riverchase',
  'Alabama',
  'Hoover',
  '33.37895802956859',
  '-86.80380210088629'],
 ['Chipotle Huntsville University Drive',
  'Alabama',
  'Huntsville',
  '34.742319254429496',
  '-86.6657204641674'],
 ['Chipotle Mobile',
  'Alabama',
  'Mobile',
  '30.675337809949887',
  '-88.143753929995'],
 ['Chipotle Westwood Plaza',
  'Alabama',
  'Mobile',
  '30.68273057569605',
  '-88.22499815689844'],
 ['Chipotle Montgomery Eastchase',
  'Alabama',
  'Montgomery',
  '32.35917687650774',
  '-86.16225285227608'],
 ['C

In [43]:
chipotle_loc_df = pd.DataFrame(locations_csv_frame[1:], columns=locations_csv_frame[0])

chipotle_loc_df.head()

,Store Title,State,City,Latitude,Longitude
0,Chipotle Auburn Campus,Alabama,Auburn,32.606812966051244,-85.48732833164195
1,Chipotle UAB Birmingham,Alabama,Birmingham,33.509721495414745,-86.80275567068401
2,Chipotle Trussville,Alabama,Birmingham,33.59558141391436,-86.64743684970284
3,Chipotle Inverness,Alabama,Birmingham,33.42258214624579,-86.69827946502971
4,Chipotle Riverchase,Alabama,Hoover,33.37895802956859,-86.80380210088629


In [44]:
cdf = chipotle_loc_df
cdf[cdf['City'] == 'San Rafael']

,Store Title,State,City,Latitude,Longitude
449,Chipotle San Rafael Montecito Plaza,California,San Rafael,37.96956572128692,-122.5167520404928
450,Chipotle San Rafael,California,San Rafael,38.0042166,-122.544202


In [49]:
cdf.groupby('State').size().sort_values(ascending=False)

State
California          414
Texas               205
Ohio                180
Florida             162
New York            153
Illinois            139
Virginia            103
Maryland             91
Pennsylvania         89
Arizona              80
Colorado             76
Minnesota            66
New Jersey           64
North Carolina       63
Massachusetts        60
Georgia              53
Missouri             39
Washington           39
Michigan             37
Indiana              37
Oregon               31
Kansas               28
Nevada               27
Connecticut          24
Tennessee            22
South Carolina       21
Washington DC        21
Kentucky             19
Wisconsin            19
Ontario              17
Alabama              14
Oklahoma             12
Utah                 11
Iowa                 10
Louisiana             9
Nebraska              9
New Mexico            8
Rhode Island          8
Delaware              8
New Hampshire         8
British Columbia      6
Arkansas  

In [56]:
cdf.to_csv(r'C:/Users/dinel/Documents/capstone_project_1/chipotle_locations.csv')